In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [65]:
from AMRGraph import AMR
from AMRData import CustomizedAMR
from utilities import pretty_print, generate_action_sequence, generate_custom_amr, generate_amr_with_literals
import ActionSequenceGenerator
import TokensReplacer
import re
import itertools
from operator import itemgetter

sentence = """The official alleged Karzai was reluctant to move against big drug lords in Karzai 's political power base in the south of Afghanistan where most opium is produced ."""
amr_string = """(a / allege-01~e.2 
      :ARG0 (p5 / person 
            :ARG0-of (h / have-org-role-91 
                  :ARG2 (o / official~e.1))) 
      :ARG1 (r / reluctant~e.5 
            :topic (m / move-04~e.7 
                  :ARG0 p 
                  :ARG1 (l / lord~e.11 
                        :mod (d / drug~e.10) 
                        :mod (b / big~e.9)) 
                  :location~e.12 (b2 / base~e.17 
                        :mod (p2 / power~e.16) 
                        :mod (p3 / politics~e.15) 
                        :poss p~e.13,14 
                        :location~e.18 (s2 / south~e.20 
                              :part-of~e.21 (c / country :wiki "Afghanistan" 
                                    :name (n / name :op1 "Afghanistan"~e.22)) 
                              :location-of~e.23 (p4 / produce-01~e.27 
                                    :ARG1 (o2 / opium~e.25 
                                          :quant (m2 / most~e.24)))))) 
            :domain~e.4 (p / person :wiki "Hamid_Karzai" 
                  :name (n3 / name :op1 "Karzai"~e.3))))
"""

In [66]:
amr = AMR.parse_string(amr_string)

In [67]:
custom_amr = generate_custom_amr(amr)


Mappings between node variables and their corresponding concepts.

{'p2': 'power', 'p3': 'politics', 'c': 'country', 'b': 'big', 'd': 'drug', 'p4': 'produce-01', 'p5': 'person', 's2': 'south', 'h': 'have-org-role-91', 'm': 'move-04', 'l': 'lord', 'o': 'official', 'n': 'name', 'p': 'person', 'r': 'reluctant', 'a': 'allege-01', 'b2': 'base', 'm2': 'most', 'n3': 'name', 'o2': 'opium'}

Mappings between nodes and all the aligned tokens: If the nodes don't havea variable (polarity, literals, quantities, interrogatives), they specify both the aligned tokens and the parent in order to uniquely identify them

{u'Karzai': [(u'3', 'n3')], u'Afghanistan': [(u'22', 'n')], 'b': ['9'], 'd': ['10'], 'p4': ['27'], 'p3': ['15'], 's2': ['20'], 'm': ['7'], 'l': ['11'], 'o': ['1'], 'p2': ['16'], 'r': ['5'], 'a': ['2'], 'b2': ['17'], 'm2': ['24'], 'o2': ['25']}

Mappings between relations and tokens. Uniquely identified by also specifying the parent of that relation.

{'domain': [[('4', 'r')]], 'part-of':

In [68]:
# get the unaligned have-org-role-91 nodes which have an ARG1 child, i.e. they can be replaced
have_org_role_nodes = [k for k in amr.node_to_concepts if amr.node_to_concepts[k] == 'have-org-role-91'
                                                          and k not in amr.node_to_tokens
                                                          and amr[k]
                                                          and "ARG2" in amr[k]]

In [69]:
have_org_role_nodes

['h']

Things to update:
- Remove have-org-role-91 from amr.node_to_concepts
- Replace the have-org-role-91 node from amr with its child having the relation "ARG1"
- Update the parent of have-org-role-91 to point to the new node

In [70]:
amr.node_to_concepts = dict((k, v) for k, v in amr.node_to_concepts.iteritems() if k not in have_org_role_nodes)

In [71]:
amr.node_to_concepts

{'a': 'allege-01',
 'b': 'big',
 'b2': 'base',
 'c': 'country',
 'd': 'drug',
 'l': 'lord',
 'm': 'move-04',
 'm2': 'most',
 'n': 'name',
 'n3': 'name',
 'o': 'official',
 'o2': 'opium',
 'p': 'person',
 'p2': 'power',
 'p3': 'politics',
 'p4': 'produce-01',
 'p5': 'person',
 'r': 'reluctant',
 's2': 'south'}

In [72]:
import copy

for h in have_org_role_nodes:
    node = amr.pop(h)
    new_node = node['ARG2']
    node.pop('ARG2')
    ### If there is both arg1 and arg2 (and arg3), it doesn't make sense
    ### to perform this operation
    if len(node) != 0:
        print "Oh noes!"
    # update the parent of have_org_role
    for k in amr:
        for rel in amr[k]:
            if amr[k][rel][0] == h:
                amr[k].replace(rel, new_node)

In [73]:
print amr

defaultdict(<class 'util.ListMap'>, {u'Afghanistan': defaultdict(<type 'list'>, {}), 'b2': defaultdict(<type 'list'>, {'location': [('s2',)], 'poss': [('p~e.13,14',)], 'mod': [('p2',), ('p3',)]}), 's2': defaultdict(<type 'list'>, {'part-of': [('c',)], 'location-of': [('p4',)]}), 'm2': defaultdict(<type 'list'>, {}), 'o2': defaultdict(<type 'list'>, {'quant': [('m2',)]}), u'Karzai': defaultdict(<type 'list'>, {}), 'a': defaultdict(<type 'list'>, {'ARG0': [('p5',)], 'ARG1': [('r',)]}), 'p2': defaultdict(<type 'list'>, {}), 'p3': defaultdict(<type 'list'>, {}), 'c': defaultdict(<type 'list'>, {'wiki': [(u'Afghanistan',)], 'name': [('n',)]}), 'b': defaultdict(<type 'list'>, {}), 'd': defaultdict(<type 'list'>, {}), 'p4': defaultdict(<type 'list'>, {'ARG1': [('o2',)]}), 'p5': defaultdict(<type 'list'>, {'ARG0-of': [('o',)]}), u'Hamid_Karzai': defaultdict(<type 'list'>, {}), 'm': defaultdict(<type 'list'>, {'ARG0': [('p',)], 'ARG1': [('l',)], 'location': [('b2',)]}), 'l': defaultdict(<type '

Apply the name entity replacer on the new amr:

In [74]:
(new_amr, new_sentence, named_entities) = TokensReplacer.replace_named_entities(amr, sentence)

In [75]:
print new_sentence

The official alleged Karzai was reluctant to move against big drug lords in Karzai 's political power base in the south of country where most opium is produced .


In [76]:
print named_entities

[('c', 'n', [u'Afghanistan'], 22, 22)]


In [77]:
print new_amr.to_amr_string()

(a / allege-01
	:ARG0 (p5 / person
		:ARG0-of (o / official))
	:ARG1 (r / reluctant
		:topic (m / move-04
			:ARG0 p
			:ARG1 (l / lord
				:mod (d / drug)
				:mod (b / big))
			:location (b2 / base
				:location (s2 / south
					:part-of (c / country)
					:location-of (p4 / produce-01
						:ARG1 (o2 / opium
							:quant (m2 / most))))
				:poss p~e.13,14
				:mod (p2 / power)
				:mod (p3 / politics)))
		:domain (p / person)))


In [64]:
print new_sentence

The official alleged Karzai was reluctant to move against big drug lords in Karzai 's political power base in the south of country where most opium is produced .
